<a href="https://colab.research.google.com/github/donggggerr/bert-spam-detection/blob/main/WithChunkingAndStratify.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
print(torch.cuda.is_available())

True


In [ ]:
import pandas as pd
from transformers import BertTokenizer
from tqdm import tqdm
import re
from bs4 import BeautifulSoup

In [ ]:
tqdm.pandas()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

df = pd.read_csv('/content/drive/MyDrive/BERT_Dataset/processed_data.csv')

In [ ]:
import re
from bs4 import BeautifulSoup

def clean_email_text(subject, message):
    text = f"{subject} {message}"

    text = BeautifulSoup(text, "html.parser").get_text()
    text = re.sub(r'content-type:.*\n', '', text, flags=re.IGNORECASE)
    text = re.sub(r'content-transfer-encoding:.*\n', '', text, flags=re.IGNORECASE)

    text = re.sub(r'\S+@\S+', '[EMAIL]', text)
    text = re.sub(r'http\S+|www\.\S+', '[LINK]', text)

    text = re.sub(r'=\n|=20|\xa0', ' ', text)
    text = text.replace('\n', ' ').replace('\r', ' ')

    text = text.lower()
    text = re.sub(r'\s+', ' ', text).strip()

    return text

In [ ]:
df['clean_email'] = df.apply(lambda row: clean_email_text(row['subject'], row['message']), axis=1)

C:\Users\rtgx2\AppData\Local\Temp\ipykernel_5812\2980855508.py:9: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  text = BeautifulSoup(text, "html.parser").get_text()


In [ ]:
df['clean_email']

0        generic cialis, branded quality@ content-trans...
1        typo in /debian/readme hi, i've just updated f...
2        authentic viagra charset="iso-8859-1" mega aut...
3        nice talking with ya hey billy, it was really ...
4        or trembling; stomach cramps; trouble in sleep...
                               ...                        
75414    the reply for your request for a job place [le...
75415    the reply for your request for a job place [le...
75416    re: [r] me again, about the horrible documenta...
75417    re: [r] rodbc problem hello, as i wrote i call...
75418    i wanted the desk at his own laws: of the. but...
Name: clean_email, Length: 75419, dtype: object

In [ ]:
from transformers import BertTokenizer

# Load BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Get token lengths
df['token_length'] = df['clean_email'].apply(lambda x: len(tokenizer.tokenize(x)))

# Show basic stats
print(df['token_length'].describe())

count     75419.000000
mean       2165.804545
std        6969.610733
min           2.000000
25%         165.000000
50%         357.000000
75%         802.000000
max      685663.000000
Name: token_length, dtype: float64


In [ ]:
def chunk_and_encode_email(text, tokenizer, max_length=512):
    tokens = tokenizer.tokenize(text)
    chunks = []

    for i in range(0, len(tokens), max_length):
        chunk_tokens = tokens[i:i + max_length]
        chunk_text = tokenizer.convert_tokens_to_string(chunk_tokens)

        encoded = tokenizer(
            chunk_text,
            padding='max_length',
            truncation=True,
            max_length=max_length,
            return_tensors='pt'
        )

        chunks.append((encoded['input_ids'], encoded['attention_mask']))

    return chunks

In [ ]:
sample_email = df['clean_email'].iloc[0]
chunks = chunk_and_encode_email(sample_email, tokenizer)

print(f"Number of chunks: {len(chunks)}")
print("Shape of first chunk input_ids:", chunks[0][0].shape)


Number of chunks: 1
Shape of first chunk input_ids: torch.Size([1, 512])


In [ ]:
sample_email = df['clean_email'].iloc[4]
chunks = chunk_and_encode_email(sample_email, tokenizer)

print(f"Number of chunks: {len(chunks)}")
print("Shape of first chunk input_ids:", chunks[0][0].shape)


Number of chunks: 19
Shape of first chunk input_ids: torch.Size([1, 512])


In [ ]:
df.head()

,label,subject,email_to,email_from,message,clean_email,token_length
0,1,"Generic Cialis, branded quality@",the00@speedy.uwaterloo.ca,"""Tomas Jacobs"" <RickyAmes@aol.com>",Content-Type: text/html;\nContent-Transfer-Enc...,"generic cialis, branded quality@ content-trans...",59
1,0,Typo in /debian/README,debian-mirrors@lists.debian.org,Yan Morin <yan.morin@savoirfairelinux.com>,"Hi, i've just updated from the gulus and I che...","typo in /debian/readme hi, i've just updated f...",180
2,1,authentic viagra,<the00@plg.uwaterloo.ca>,"""Sheila Crenshaw"" <7stocknews@tractionmarketin...","Content-Type: text/plain;\n\tcharset=""iso-8859...","authentic viagra charset=""iso-8859-1"" mega aut...",95
3,1,Nice talking with ya,opt4@speedy.uwaterloo.ca,"""Stormy Dempsey"" <vqucsmdfgvsg@ruraltek.com>","Hey Billy, \n\nit was really fun going out the...","nice talking with ya hey billy, it was really ...",154
4,1,or trembling; stomach cramps; trouble in sleep...,ktwarwic@speedy.uwaterloo.ca,"""Christi T. Jernigan"" <dcube@totalink.net>",Content-Type: multipart/alternative;\n ...,or trembling; stomach cramps; trouble in sleep...,9367


In [ ]:
all_input_ids = []
all_attention_masks = []
all_labels = []
email_chunk_map = []  # NEW

for idx, row in tqdm(df.iterrows(), total=len(df)):
    chunks = chunk_and_encode_email(row['clean_email'], tokenizer)

    for input_ids, attention_mask in chunks:
        all_input_ids.append(input_ids)
        all_attention_masks.append(attention_mask)
        all_labels.append(row['label'])  # same label for every chunk
        email_chunk_map.append(idx)  # track which email this chunk came from


100%|████████████████████████████████████████████████████████████████████████████| 75419/75419 [46:02<00:00, 27.30it/s]


In [ ]:
print(f"Total chunks: {len(all_input_ids)}")


Total chunks: 359773


In [ ]:
import torch

input_ids_tensor = torch.cat(all_input_ids, dim=0)
attention_masks_tensor = torch.cat(all_attention_masks, dim=0)
labels_tensor = torch.tensor(all_labels)


In [ ]:
print("Input IDs shape:", input_ids_tensor.shape)
print("Attention mask shape:", attention_masks_tensor.shape)
print("Labels shape:", labels_tensor.shape)


Input IDs shape: torch.Size([359773, 512])
Attention mask shape: torch.Size([359773, 512])
Labels shape: torch.Size([359773])


In [ ]:
from sklearn.model_selection import train_test_split

train_inputs, val_inputs, train_masks, val_masks, train_labels, val_labels = train_test_split(
    input_ids_tensor,
    attention_masks_tensor,
    labels_tensor,
    test_size=0.2,
    stratify=labels_tensor,
    random_state=42
)


In [ ]:
from sklearn.model_selection import train_test_split

# Do the same split again, but now for email_chunk_map
_, val_email_ids = train_test_split(
    email_chunk_map,
    test_size=0.2,
    stratify=labels_tensor,
    random_state=42
)

In [ ]:
from torch.utils.data import TensorDataset, DataLoader

train_dataset = TensorDataset(train_inputs, train_masks, train_labels)
val_dataset = TensorDataset(val_inputs, val_masks, val_labels)


In [ ]:
batch_size = 16

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)


In [ ]:
from transformers import BertForSequenceClassification

model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [ ]:
from torch.optim import AdamW

optimizer = AdamW(model.parameters(), lr=2e-5)


In [ ]:
from tqdm import tqdm

model.train()

for batch in tqdm(train_loader):
    input_ids, attention_mask, labels = [x.to(device) for x in batch]

    outputs = model(
        input_ids=input_ids,
        attention_mask=attention_mask,
        labels=labels
    )

    loss = outputs.loss
    loss.backward()

    optimizer.step()
    optimizer.zero_grad()


100%|██████████████████████████████████████████████████████████████████████████| 17989/17989 [5:53:09<00:00,  1.18s/it]


In [ ]:
from torch.nn.functional import softmax

model.eval()
all_chunk_probs = []

with torch.no_grad():
    for input_ids, mask, labels in val_loader:
        input_ids = input_ids.to(device)
        mask = mask.to(device)

        outputs = model(input_ids=input_ids, attention_mask=mask)
        probs = softmax(outputs.logits, dim=1)[:, 1]  # spam class
        all_chunk_probs.extend(probs.cpu().tolist())


In [ ]:
from collections import defaultdict
import numpy as np

email_scores = defaultdict(list)
email_true_labels = {}

for prob, email_id, label in zip(all_chunk_probs, val_email_ids, val_labels):
    email_scores[email_id].append(prob)
    email_true_labels[email_id] = label.item()


In [ ]:
final_preds = []
final_labels = []

for email_id, scores in email_scores.items():
    avg_score = np.mean(scores)
    pred = 1 if avg_score >= 0.5 else 0

    final_preds.append(pred)
    final_labels.append(email_true_labels[email_id])


In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

print("Accuracy:", accuracy_score(final_labels, final_preds))
print("Precision:", precision_score(final_labels, final_preds))
print("Recall:", recall_score(final_labels, final_preds))
print("F1:", f1_score(final_labels, final_preds))


Accuracy: 0.9933533608711979
Precision: 0.9918167809129071
Recall: 0.9987692636986302
F1: 0.9952808809022315


In [ ]:
model.save_pretrained("long_training_my_spam_model")
tokenizer.save_pretrained("long_training_my_spam_model")

('long_training_my_spam_model\\tokenizer_config.json',
 'long_training_my_spam_model\\special_tokens_map.json',
 'long_training_my_spam_model\\vocab.txt',
 'long_training_my_spam_model\\added_tokens.json')